# Exogenous Processes

In models without exogenous processes the law of motion, the transition of states over periods, is deterministic. Given a state and a choice the state in the next period is certain.

Exogenous processes introduce a stochastic element to the law of motion. Given a state and a choice there exist multiple states in the next period which are valid successors. The transition to one of the successors is determined by a probability which depends on the characteristics of the state.

In [1]:
import pandas as pd
import numpy as np
import respy as rp

pd.set_option("display.max_rows", 100)

## A single exogenous process

In [2]:
params, options = rp.get_example_model("robinson_crusoe_basic", with_data=False)

In [3]:
params.loc[("nonpec_fishing", "sick"), "value"] = -2

In [4]:
params.loc[("exogenous_process_illness_sick", "probability"), "value"] = 0.1
params.loc[("exogenous_process_illness_healthy", "probability"), "value"] = 0.9

In [5]:
options["covariates"]["sick"] = "illness == 'sick'"

In [6]:
solve = rp.get_solve_func(params, options)

In [7]:
state_space = solve(params)

In [8]:
simulate = rp.get_simulate_func(params, options)

In [9]:
df = simulate(params)

In [10]:
df["Prev_Illness"] = df.groupby("Identifier").Illness.shift()
pd.crosstab(df.Illness, df.Prev_Illness, normalize=True).to_numpy()

array([[0.81655556, 0.08633333],
       [0.087     , 0.01011111]])

In [11]:
loglike = rp.get_crit_func(params, options, df)

In [12]:
loglike(params)

-14.763326170625826

## Exogenous process affecting availability of choices

In [13]:
params.loc[("exogenous_process_weather_tropical_storm", "probability"), "value"] = 0.2
params.loc[("exogenous_process_weather_sunny", "probability"), "value"] = 0.8

In [14]:
options["inadmissible_states"] = {
    "fishing": ["weather == 'tropical_storm'"]
}

In [15]:
solve = rp.get_solve_func(params, options)

In [16]:
state_space = solve(params)

In [18]:
simulate = rp.get_simulate_func(params, options)

In [19]:
df = simulate(params)

In [21]:
pd.crosstab(df.Choice, df.Weather, normalize=True)

Weather,sunny,tropical_storm
Choice,,
fishing,0.1359,0.0000
hammock,0.6598,0.2043


## References

> Behrman, J. R., Tincani, M. M., Todd, P. E., & Wolpin, K. I. (2016). [Teacher quality in public and private schools under a voucher system: The case of Chile.](https://scholarship.rice.edu/bitstream/handle/1911/94185/Teacher-Quality-Chile.pdf) *Journal of Labor Economics, 34(2)*, 319-362.